# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [6]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can create high-quality, engaging content such as blog posts, social media posts, product descriptions, and even entire articles. This can help reduce the workload on human writers and editors.
2. **Image and Video Generation**: Generative AI can generate high-quality images and videos that can be used for marketing campaigns, advertising, and other visual content needs.
3. **Chatbots and Virtual Assistants**: Generative AI can power chatbots and virtual assistants that provide customer support, answer frequently asked questions, and offer personalized recommendations.
4. **Recommendation Systems**: Generative AI can analyze user behavior and preferences to generate personalized product recommendations on e-commerce websites, streaming services, and other platforms.
5. **Predictive Maintenance**: Generative AI can analyze sensor data from equipme

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [7]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can be used to create personalized content, such as articles, social media posts, and product descriptions, saving time and resources for businesses.
2. **Marketing Automation**: Generative AI can generate targeted marketing campaigns, including email templates, ad copy, and promotional materials, helping businesses personalize their marketing efforts.
3. **Product Design**: Generative AI can be used to design new products, such as furniture, clothing, and electronics, reducing the time and cost associated with traditional design processes.
4. **Image and Video Generation**: Generative AI can create realistic images and videos, which can be used in various applications, such as advertising, entertainment, and e-learning.
5. **Data Analysis**: Generative AI can analyze large datasets and identify patterns and trends, helping businesses make data-dri

## Alternative approach - using OpenAI python library to connect to Ollama

In [8]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: AI can generate high-quality content such as articles, social media posts, product descriptions, and even entire books. This allows businesses to reduce content creation costs, improve consistency and quality, and scale their content output.
2. **Marketing and Advertising**: Generative AI can help create personalized ads, product descriptions, and promotional materials that resonate with specific audiences. It can also generate product demos, walkthroughs, and training videos.
3. **Product Design and Development**: AI-powered generative design tools can rapidly prototype and iterate on new product designs, reducing the time and cost associated with traditional design processes.
4. **Customer Service Chatbots**: Generative AI can power chatbots that provide 24/7 customer support, offer personalized recommendations, and help with simple queries, freeing up human customer suppo

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [9]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 3.0 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 4.5 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 8.0 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  11 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  13 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  18 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  23 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  23 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕       

In [10]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

<think>
Okay, so I need to write about the definitions of some core concepts related to large language models (LLMs), specifically including neural networks, attention, and transformers. Hmm, let's break this down. 

First, I know that neural networks are like biological models in their intelligence, but I remember they're computational systems inspired by the brain. They process information through layers of neurons. Now, beyond computations, LLMs take text input and produce outputs. That’s probably a key point about how NNs function in LLMs.

Attention seems to be a mechanism for focusing the model's output on specific parts of the input. I think it was introduced by Google in 2015 with their "attention mechanisms." It's like using self-attentional layers, which process each part of the text and highlight the important words or phrases.

Transformers are supposed to be scalable and efficient without using traditional neural network layers. Their main idea is that they can handle diff

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [23]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [24]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [29]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [25]:
import ollama

#response = ollama.chat(model=MODEL, messages=messages)
#print(response['message']['content'])

In [26]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [32]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [33]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [34]:
# And now: call the Ollama function instead of OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [35]:
summarize("https://edwarddonner.com")

'**Summary of the Website**\n\n### About the Creator\n\n* Ed Donner is a writer, code enthusiast, and DJ who also founded Nebula.io, an AI company.\n* He enjoys amateur electronic music production, reading Hacker News, and losing himself in technology.\n\n### Company Overview\n\n* **Nebula.io**: An AI company that applies machine learning to help people discover their potential. \n* The company offers a product used by recruiters to source, understand, engage with, and manage talent.\n* Nebula has patented its matching model and won press coverage.\n\n### Recent Announcements\n\n* **Complete Agentic AI Engineering Course**: Launched on April 21, 2025\n* **LLM Workshop – Hands-on with Agents – resources**: Launched on December 21, 2024\n* **Welcome, SuperDataScientists!**: Published on November 13, 2024\n* **Mastering AI and LLM Engineering – Resources**: Published in January 2025 (exact date not specified)\n\n### Social Media Links\n\n* LinkedIn: Follow him for updates.\n* Twitter: Fol

In [21]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [36]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

**Overview**
Edward Donner is a co-founder and CTO of Nebula.io, an AI startup that applies AI to help people discover their potential. He is also the founder and former CEO of another AI startup, untapt.

### News/Announcements

* **The Complete Agentic AI Engineering Course**: Announced for April 21, 2025
* **LLM Workshop – Hands-on with Agents – resources**: Released on December 21, 2024
* **Mastering AI and LLM Engineering – Resources**: Launched in November 2024
* **Welcome, SuperDataScientists!**: Introduced in November 2024